In [8]:
from transformers import BertTokenizer, AutoConfig, AutoModelForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-offline", num_labels=2)

HTTPError: 500 Server Error: Internal Server Error for url: https://huggingface.co/bert-base-cased/resolve/main/added_tokens.json

In [ ]:
from glob import glob
from datasets import Dataset

import re
import pandas as pd

def data_loading(url):
    with open(url, 'r', encoding='utf-8') as f:
        # data = json.loads(f.read())
        df = pd.read_json(f)
        data = df.copy()
        # data = df.loc[:, ['sentiment', 'body']]
        data = data.loc[df['sentiment'].notnull()]
        data['sentiment'] = pd.Categorical(data['sentiment'])
        data['label'] = data['sentiment'].cat.codes
        data = data.rename(columns={'sentiment': 'labels', 'body': 'sentense'})

        return data

data_url = '../crawler/stock/data/**.json'
url = glob(data_url)[-1]
data = data_loading(url)

dataset = Dataset.from_pandas(data.loc[:, ['label', 'sentense']])
dataset = dataset.remove_columns('__index_level_0__')

In [ ]:
def encode(example):
    result = tokenizer(example['sentense'], padding='max_length', truncation=True)
    
    return result

encoded_dataset = dataset.map(encode, batched=True)
encoded_dataset = encoded_dataset.train_test_split(test_size=0.2)
# print(encoded_dataset[0]['sentense'])
# print(encoded_dataset[0]['input_ids'])
# type(encoded_dataset[0]['labels'])

In [ ]:
from transformers import TrainingArguments, Trainer
from datasets import load_metric

import numpy as np
import wandb

%env WANDB_PROJECT=Stocktwit_sentiment_analysis
training_args = TrainingArguments(
    output_dir="bert-classifier-trainer",
    evaluation_strategy="step",
    eval_steps=50,
    num_train_epochs=1,
    save_steps=1000,
    per_device_train_batch_size=4,
    report_to="wandb",
    run_name="bert-sentiment-classifier-test"
)

metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    compute_metrics=compute_metrics,
)

trainer.train()


wandb.finish()

env: WANDB_PROJECT=Stocktwit_sentiment_analysis


In [ ]:
model.save_pretrained('./bert-classifier-trainer/test-epoch-1')